In [1]:
import os

os.environ['KERAS_BACKEND'] = "torch"

# Dataset Generation
Our dataset is a corpus of reviews scrapped from the BGG API. <br /> 
In order to download the comments we make use of the ```bgg_corpus_service.py``` content.

BGG Does not directly provide a way to list all the games it has in archive therefore we used a dump created by the community (2024-08-18).

In [2]:
# /resources/2024-08-18.csv
corpus_file = "../data/corpus.preprocessed.csv"

In [7]:
import core.utils as utils
import core.embeddings as embeddings

embeddings_model = embeddings.WordEmbedding(
    utils.LoadCorpusUtility(), max_vocab_size=10000, embedding_size=128,
    target_model_file="./../data/word-embeddings.model", corpus_file=corpus_file
)

# Custom Dataloaders Definitions
To train the model we require a way to get elements of our dataset. I designed 3 classes for this:

Without preprocessing on dataset: 379211268 <br />
If the dataset is processed already: 677671688 (Twice the size) <br />

# PositiveNegativeCommentGeneratorDataset

In [8]:
from core.dataset import PositiveNegativeCommentGeneratorDataset

embeddings_model.load_model()
embeddings_model.get_vocab()
vocabulary = embeddings_model.model.wv.key_to_index

ds = PositiveNegativeCommentGeneratorDataset("./../data/corpus.preprocessed.csv", vocabulary, 10)

INFO:gensim.utils:loading Word2Vec object from ../data/word-embeddings.model
DEBUG:smart_open.smart_open_lib:{'uri': '../data/word-embeddings.model', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'compression': 'infer_from_extension', 'transport_params': None}
INFO:gensim.utils:loading wv recursively from ../data/word-embeddings.model.wv.* with mmap=None
INFO:gensim.utils:setting ignored attribute cum_table to None
INFO:gensim.utils:Word2Vec lifecycle event {'fname': '../data/word-embeddings.model', 'datetime': '2024-11-20T16:37:42.477190', 'gensim': '4.3.3', 'python': '3.12.3 (main, Nov  6 2024, 18:32:19) [GCC 13.2.0]', 'platform': 'Linux-6.8.0-48-generic-x86_64-with-glibc2.39', 'event': 'loaded'}


Pandas Apply:   0%|          | 0/1000 [00:00<?, ?it/s]

In [9]:
from torch.utils.data import DataLoader

lazy_dataloader = DataLoader(ds, batch_size=32, shuffle=True, collate_fn=lambda x: x)